In [94]:
%%capture
!pip install selenium

In [95]:
%%capture
!pip install webdriver-manager

In [11]:
# Importing All the Important Libraries 
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [12]:
Dates=pd.read_excel(r"./Dates.xlsx")

In [13]:
# Cross Check Extracting Dates 
Dates['week']=np.where(Dates['Month'].isin(["Jan","May"]),Dates['week']+1,Dates['week'])
Dates=Dates[(Dates['year']==2023) & (Dates['Month']=='Jan')]
Dates.head(10)

,Date,Month,year,Day,Day_in_month,DayofWeek,week
59,2023-01-31,Jan,2023,Tuesday,31,3,6
60,2023-01-30,Jan,2023,Monday,30,2,6
61,2023-01-29,Jan,2023,Sunday,29,1,6
62,2023-01-28,Jan,2023,Saturday,28,7,5
63,2023-01-27,Jan,2023,Friday,27,6,5
64,2023-01-26,Jan,2023,Thursday,26,5,5
65,2023-01-25,Jan,2023,Wednesday,25,4,5
66,2023-01-24,Jan,2023,Tuesday,24,3,5
67,2023-01-23,Jan,2023,Monday,23,2,5
68,2023-01-22,Jan,2023,Sunday,22,1,5


In [14]:
# Website which we will be using 
website='https://vegetablemarketprice.com/market/noida/today'

In [15]:
# For getting into each Date
Data_list=[]
driver = webdriver.Chrome(options=webdriver.ChromeOptions() ) # opening the Browser
sleep(1)
driver.maximize_window() # Maximizing the windows
driver.get(website)     # Loading the Website
sleep(1)
for index in Dates.index:
    Desired_month= str(Dates['Month'][index]+ ' '+ str(Dates['year'][index]))
    week_row=Dates['week'][index]
    Day_cell=Dates['DayofWeek'][index]
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[1]").click()  #Calendar dropdown
    sleep(1)
    while driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[2]").text != Desired_month :
        driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[1]").click()
        sleep(0.1)
    driver.find_element(By.XPATH,f'/html/body/div[4]/div[2]/div[1]/table/tbody/tr[{week_row}]/td[{Day_cell}]').click() # Date Select 
    try:
        myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'IdOfMyElement')))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
    elements =driver.find_elements(By.CLASS_NAME,"todayVetableTableRows")
    sleep(1)
    for element in elements:
        id=element.get_attribute('id')
        Vegetable=driver.find_element(By.XPATH,'//*[@id="{}"]/td[2]'.format(str(id)))
        Wholesale=driver.find_element(By.XPATH,'//*[@id="{}"]/td[3]'.format(str(id)))
        Retail=   driver.find_element(By.XPATH,'//*[@id="{}"]/td[4]'.format(str(id)))
        Shopping= driver.find_element(By.XPATH,'//*[@id="{}"]/td[5]'.format(str(id)))
        Units=    driver.find_element(By.XPATH,'//*[@id="{}"]/td[6]'.format(str(id)))
        Date=     Dates['Date'][index]
        Data_list.append({'Date': Date,
            'Vegetable':Vegetable.text,
              'Wholesale Price':Wholesale.text,
              'Retail Price':Retail.text,
              'Shopping Mall Price':Shopping.text,
              'Units':Units.text})
        sleep(0.01)
print("Scrapping Done")

Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Loading took too much time!
Scrapping Done


In [16]:
df=pd.DataFrame(Data_list)

In [17]:
# Checking How many Days Data We got 
df

,Date,Vegetable,Wholesale Price,Retail Price,Shopping Mall Price,Units
0,2023-01-31,Onion Big,₹25,₹29 - 32,₹30 - 41,1kg
1,2023-01-31,Onion Small,₹37,₹43 - 47,₹44 - 61,1kg
2,2023-01-31,Tomato,₹26,₹30 - 33,₹31 - 43,1kg
3,2023-01-31,Green Chilli,₹38,₹44 - 48,₹46 - 63,1kg
4,2023-01-31,Beetroot,₹36,₹41 - 46,₹43 - 59,1kg
...,...,...,...,...,...,...
1669,2023-01-01,Shallot (Pearl Onion),₹36,₹41 - 46,₹43 - 59,1kg
1670,2023-01-01,Snake Gourd,₹28,₹32 - 36,₹34 - 46,1kg
1671,2023-01-01,Sorrel Leaves,₹12,₹14 - 15,₹14 - 20,1kg
1672,2023-01-01,Spinach,₹13,₹15 - 17,₹16 - 21,1kg


In [18]:
# Data Cleaning And Pre Processing and Converting it into Different 
df["Wholesale Price"]=df["Wholesale Price"].str.replace('₹','').astype(np.int64)
df["Retail Price"]=df["Retail Price"].str.replace('₹','')
df["Shopping Mall Price"]=df["Shopping Mall Price"].str.replace('₹','')
df["Retail Price"]=(df["Retail Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Retail Price"].str.split('-',expand=True)[0].astype(np.int64) )//2
df["Shopping Mall Price"]=(df["Shopping Mall Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Shopping Mall Price"].str.split('-',expand=True)[0].astype(np.int64) )//2

In [19]:
# Cross Checking did we get proper data or not 
df

,Date,Vegetable,Wholesale Price,Retail Price,Shopping Mall Price,Units
0,2023-01-31,Onion Big,25,30,35,1kg
1,2023-01-31,Onion Small,37,45,52,1kg
2,2023-01-31,Tomato,26,31,37,1kg
3,2023-01-31,Green Chilli,38,46,54,1kg
4,2023-01-31,Beetroot,36,43,51,1kg
...,...,...,...,...,...,...
1669,2023-01-01,Shallot (Pearl Onion),36,43,51,1kg
1670,2023-01-01,Snake Gourd,28,34,40,1kg
1671,2023-01-01,Sorrel Leaves,12,14,17,1kg
1672,2023-01-01,Spinach,13,16,18,1kg


In [20]:
# Change File Name Accoring to Choice
df.to_csv('NoidaData1.csv')